## 1. Transform Grayscale image to specific RGB mappings

In [1]:
import os.path as osp
import os
import errno
import cv2
from matplotlib import pyplot as plt
import numpy as np
import json
from collections import namedtuple

In [2]:
root_path = "S2DSdevkit"
part_path = "part2"
images_path = "images"

def createValidPathnames(root_path, part_path, images_path):
    """
        returns a tuple containing the three needed pathnames
        contains a check if the file is there
    """
    labelsGrayscale_path = osp.join(root_path, part_path, "labelsGrayscale")
    if os.path.isdir(labelsGrayscale_path):
        print("Found grayscale label folder.")
    else:
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), labelsGrayscale_path)

    # labels RGB path
    labelsRGB_path = osp.join(root_path, part_path, "labelsRGB")
    if os.path.isdir(labelsRGB_path):
        print("Found RGB label folder.")
    else:
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), labelsRGB_path)

    # color mapping path
    colorMapping_path = osp.join(root_path, part_path, "ColorMapping.json" )
    if os.path.isdir(labelsGrayscale_path):
        print("Found color mapping file.")
    else:
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), colorMapping_path)
    return labelsGrayscale_path, labelsRGB_path, colorMapping_path

labelsGrayscale_path, labelsRGB_path, colorMapping_path = createValidPathnames(root_path, part_path, images_path)

Found grayscale label folder.
Found RGB label folder.
Found color mapping file.


In [4]:
def createColorMapping(json_path):
    """
        reads a json file and creates a list of object containing the transformation details
        eg.
        colorMapping = [ColorMapping(Class='Spalling', RGB=[255, 0, 0], Grayscale=25),
         ColorMapping(Class='Rust Stain', RGB=[165, 42, 42], Grayscale=120), ... ]
         Access the element:
         colorMapping[0].RGB -> [255,0,0]
    """
    colorMapping = []
    with open(colorMapping_path) as json_file:
        json_file = json_file.read()
        json_file = json_file.split(";")
        for data in json_file:
            colorMapping.append(json.loads(data, object_hook=lambda d: namedtuple('ColorMapping', d.keys())(*d.values())))
    return colorMapping

colorMapping = createColorMapping(colorMapping_path)
colorMapping

[ColorMapping(Class='Spalling', RGB=[255, 0, 0], Grayscale=25),
 ColorMapping(Class='Rust Stain', RGB=[165, 42, 42], Grayscale=120),
 ColorMapping(Class='Discoloration', RGB=[0, 255, 255], Grayscale=127),
 ColorMapping(Class='Vegatation', RGB=[127, 255, 0], Grayscale=50),
 ColorMapping(Class='Crack', RGB=[255, 255, 255], Grayscale=159),
 ColorMapping(Class='Graffiti', RGB=[128, 0, 128], Grayscale=90)]

In [5]:
# create list of files in the folder
list_paths = os.listdir(labelsRGB_path)
# get only images ending on .jpg
list_labels_rgb_with_postfix = list((filter(lambda k: '.png' in k, list_paths)))
# remove the file ending .jpg
list_labels_rgb_without_postfix = list((map(lambda k: osp.splitext(k)[0], list_labels_rgb_with_postfix)))
# show first five elements
list_labels_rgb_without_postfix[:5]

['01 (405)', 'IMG_0798', 'IMG_7393', '02', '01 (45)']

In [ ]:
"""
# test with one img
origin_label_1_path = osp.join(labelsRGB_path, list_labels_rgb_without_postfix[0] + ".png")
target_label_1_path = osp.join(labelsGrayscale_path, list_labels_rgb_without_postfix[0] + ".png")
img = cv2.imread(origin_label_1_path)
"""

In [8]:
def RGBtoGray(RGBimg, colorMapping):
    RGBimg = cv2.cvtColor(RGBimg, cv2.COLOR_BGR2RGB)
    row = RGBimg.shape[0]
    col = RGBimg.shape[1]
    GRAYimg = np.zeros((row, col))
    for x in range(0,row):
        for y in range(0,col):
            pixel = RGBimg[x,y,:]
            for cm in colorMapping:
                if np.array_equal(pixel, np.array(cm.RGB)):
                    GRAYimg[x,y] = cm.Grayscale    
    return GRAYimg    

In [11]:
for name_img in list_labels_rgb_without_postfix:
    print("Processing image " + name_img + "...", end = '')
    origin_label_path = osp.join(labelsRGB_path, name_img + ".png")
    target_label_path = osp.join(labelsGrayscale_path, name_img + ".png")
    image = cv2.imread(origin_label_path)
    gray = RGBtoGray(image, colorMapping)
    cv2.imwrite(target_label_path, gray)
    print(" ... finished!")

Processing image 01 (405)...
 ... finished!Processing image IMG_0798...


KeyboardInterrupt: 